In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
import plotly.express as px
from pyoso import Client

load_dotenv()
OSO_API_KEY = os.environ['OSO_API_KEY']
client = Client(api_key=OSO_API_KEY)

In [2]:
def stringify(arr):
    return "'" + "','".join(arr) + "'"

In [3]:
PROJECT_NAMES = [
    'eth-infinitism-account-abstraction', #OSSD
    '0xb98778ca9ff41446e2bc304f7b5d27f0fa7c2bcd11df19e22d1352c06698a1f6',
    
    'cyberconnecthq',
    
    'across',
    '0x72723e07fe409557489a6643b43d9493a94c10ba68230b0527f01834cb6a550f',

    'velodrome',
    '0x08df6e20a3cfabbaf8f34d4f4d048fe7da40447c24be0f3ad513db6f13c755dd',

    'aerodrome-finance',
    '0x4288aacdda65af69c03b2c9b9d057e2b93ed1b7671e481b03c6a7b33349d2365',

    'aave',
    '0x3061b642db56c507e265f03029735b0413a613bda43456a31e78128baeb18754'
]
START_DATE = '2025-02-01'
END_DATE = '2025-03-01'

In [4]:
df_projects = client.to_pandas(f"""
select project_id from oso.projects_v1
where project_name in ({stringify(PROJECT_NAMES)})
""")
PROJECT_IDS = list(df_projects['project_id'].unique())
PROJECT_IDS

['MSfDkUunLKtfuMRdZuSxuemESQuIWKFCh1O6ySIEUs8=',
 'VsbnIqH0Z8ErW7VixsMTGcLVf45E645bcxAVcD/KZT0=',
 'inHPXAwcpnmNnzuMDBO9CsDuph+M4UNF3XwNNhTpafs=',
 'bHGrs5wFQsPVTRZ9AALMgCizZwlL1BrL6iFLaIo9F2k=',
 'hE1BjGS8I9OXjnYATSkRnNJlsItBXO+CKXU6UPKJlRQ=',
 'PjPqPTput/vvThUxx6bMhInMKHuCV3t1s+K4YiHjIRg=',
 'ZtwRwweUfJxkM3rtVoIfvvOaqHRc7CGlSCAKE8VgmN0=',
 '1tglFLCPO1i0woe4TW7pBm1iMqrhlWJ1cv2L/5b0VUk=',
 'jVDQDrrV0r8aRWDzxJdVtTLUvjmaqMDE8SynmtPND9Y=',
 '5/9fRbVoDF7pHwBdiMbNUbKhidEt/12Ojt9f8cWoZnQ=',
 'rxrN8HnLLV8SOP3md3NfXpntywh7F7n5fuRMqFB3Upo=']

In [5]:
df_superchain_events = client.to_pandas(f"""
select
    project_id,
    sum(gas_fee) as gas_fees
from oso.int_superchain_events_by_project
where
    event_type = 'CONTRACT_INVOCATION'
    and project_id IN ({stringify(PROJECT_IDS)})
    and time >= DATE('{START_DATE}') AND time < DATE('{END_DATE}')
group by 1
order by 2 desc
""")
df_superchain_events.head()

,project_id,gas_fees
0,rxrN8HnLLV8SOP3md3NfXpntywh7F7n5fuRMqFB3Upo=,187.705441
1,ZtwRwweUfJxkM3rtVoIfvvOaqHRc7CGlSCAKE8VgmN0=,187.705441
2,1tglFLCPO1i0woe4TW7pBm1iMqrhlWJ1cv2L/5b0VUk=,69.685458
3,MSfDkUunLKtfuMRdZuSxuemESQuIWKFCh1O6ySIEUs8=,69.685458
4,5/9fRbVoDF7pHwBdiMbNUbKhidEt/12Ojt9f8cWoZnQ=,47.49231


In [6]:
df_blockchain_events = client.to_pandas(f"""
select
    abp.project_id,
    sum(e.amount/1e18) as gas_fees
from oso.int_events_daily__blockchain e
join oso.artifacts_by_project_v1 abp on e.to_artifact_id = abp.artifact_id
where
    e.event_type = 'CONTRACT_INVOCATION'
    and abp.project_id IN ({stringify(PROJECT_IDS)})
    and e.bucket_day >= DATE('{START_DATE}') AND e.bucket_day < DATE('{END_DATE}')
group by 1
order by 2 desc
""")
df_blockchain_events.head()

,project_id,gas_fees
0,ZtwRwweUfJxkM3rtVoIfvvOaqHRc7CGlSCAKE8VgmN0=,187.705441
1,rxrN8HnLLV8SOP3md3NfXpntywh7F7n5fuRMqFB3Upo=,187.705441
2,MSfDkUunLKtfuMRdZuSxuemESQuIWKFCh1O6ySIEUs8=,69.685458
3,1tglFLCPO1i0woe4TW7pBm1iMqrhlWJ1cv2L/5b0VUk=,69.685458
4,5/9fRbVoDF7pHwBdiMbNUbKhidEt/12Ojt9f8cWoZnQ=,47.49231


In [7]:
df_metrics_by_artifact = client.to_pandas(f"""
select
    abp.project_id,
    sum(tm.amount) as gas_fees
from oso.timeseries_metrics_by_artifact_v0 tm
join oso.metrics_v0 m on tm.metric_id = m.metric_id
join oso.artifacts_by_project_v1 abp on tm.artifact_id = abp.artifact_id
where    
    tm.sample_date >= DATE('{START_DATE}') AND tm.sample_date < DATE('{END_DATE}')    
    and abp.project_id IN ({stringify(PROJECT_IDS)})
    and m.metric_name LIKE '%_gas_fees_daily'
group by 1
order by 2 desc
""")
df_metrics_by_artifact.head()

,project_id,gas_fees
0,ZtwRwweUfJxkM3rtVoIfvvOaqHRc7CGlSCAKE8VgmN0=,187.705441
1,rxrN8HnLLV8SOP3md3NfXpntywh7F7n5fuRMqFB3Upo=,187.705441
2,1tglFLCPO1i0woe4TW7pBm1iMqrhlWJ1cv2L/5b0VUk=,69.685458
3,MSfDkUunLKtfuMRdZuSxuemESQuIWKFCh1O6ySIEUs8=,69.685458
4,5/9fRbVoDF7pHwBdiMbNUbKhidEt/12Ojt9f8cWoZnQ=,47.49231


In [8]:
df_metrics_by_project = client.to_pandas(f"""
select
    tm.project_id,
    sum(tm.amount) as gas_fees
from oso.timeseries_metrics_by_project_v0 tm
join oso.metrics_v0 m on tm.metric_id = m.metric_id
where    
    tm.sample_date >= DATE('{START_DATE}') AND tm.sample_date < DATE('{END_DATE}')    
    and tm.project_id IN ({stringify(PROJECT_IDS)})
    and m.metric_name LIKE '%_gas_fees_daily'
group by 1
order by 2 desc
""")
df_metrics_by_project.head()

,project_id,gas_fees
0,rxrN8HnLLV8SOP3md3NfXpntywh7F7n5fuRMqFB3Upo=,187.705441
1,ZtwRwweUfJxkM3rtVoIfvvOaqHRc7CGlSCAKE8VgmN0=,187.705441
2,MSfDkUunLKtfuMRdZuSxuemESQuIWKFCh1O6ySIEUs8=,69.685458
3,1tglFLCPO1i0woe4TW7pBm1iMqrhlWJ1cv2L/5b0VUk=,69.685458
4,5/9fRbVoDF7pHwBdiMbNUbKhidEt/12Ojt9f8cWoZnQ=,47.49231


In [9]:
tables = {
    'superchain_events': df_superchain_events,
    'blockchain_events': df_blockchain_events,
    'timeseries_artifact': df_metrics_by_artifact,
    'timeseries_project': df_metrics_by_project
}
df_combined = pd.concat([
    dataframe.set_index('project_id').rename(columns={'gas_fees': label})
    for (label, dataframe) in tables.items()
], axis=1)

df_combined

,superchain_events,blockchain_events,timeseries_artifact,timeseries_project
project_id,,,,
rxrN8HnLLV8SOP3md3NfXpntywh7F7n5fuRMqFB3Upo=,187.705441,187.705441,187.705441,187.705441
ZtwRwweUfJxkM3rtVoIfvvOaqHRc7CGlSCAKE8VgmN0=,187.705441,187.705441,187.705441,187.705441
1tglFLCPO1i0woe4TW7pBm1iMqrhlWJ1cv2L/5b0VUk=,69.685458,69.685458,69.685458,69.685458
MSfDkUunLKtfuMRdZuSxuemESQuIWKFCh1O6ySIEUs8=,69.685458,69.685458,69.685458,69.685458
5/9fRbVoDF7pHwBdiMbNUbKhidEt/12Ojt9f8cWoZnQ=,47.49231,47.49231,47.49231,47.49231
bHGrs5wFQsPVTRZ9AALMgCizZwlL1BrL6iFLaIo9F2k=,47.104399,47.104399,47.104399,47.104399
PjPqPTput/vvThUxx6bMhInMKHuCV3t1s+K4YiHjIRg=,10.905293,10.905293,10.905293,10.905293
inHPXAwcpnmNnzuMDBO9CsDuph+M4UNF3XwNNhTpafs=,5.500691,5.500691,5.500691,5.500691
VsbnIqH0Z8ErW7VixsMTGcLVf45E645bcxAVcD/KZT0=,2.01175,2.01175,2.01175,2.01175
